In [43]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer,CountVectorizer
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType

spark = SparkSession.Builder().appName('SparkML Sentiment Analysis').getOrCreate()
yorumDF = spark.read.option('inferSchema','true')\
                        .option('header','false')\
                        .option('delimiter','\t').csv('datasets/movie_turkish_train.txt')
yorumDF = yorumDF.withColumnRenamed('_c0','cumle')
yorumDF = yorumDF.withColumnRenamed('_c1','label')

def fixCumle(cumle):
    cumle = cumle.replace('1','bir')
    #cumle = cumle.replace('.','')
    #cumle = cumle.replace('(','')
    #cumle = cumle.replace(')','')
    #cumle = cumle.strip()
    #cumle = cumle.replace('filmi','film')
    return cumle

fixTextUDF = UserDefinedFunction(fixCumle,StringType())
yorumDF = yorumDF.withColumn('cumle2',fixTextUDF('cumle'))

tknzr = Tokenizer(inputCol='cumle2',outputCol='tokenized')
yorumDF = tknzr.transform(yorumDF)
#yorumDF.show()

kelimeSayisi = 3500
vectorizer = CountVectorizer(inputCol='tokenized',outputCol='features',vocabSize=kelimeSayisi)
vecModel = vectorizer.fit(yorumDF)

yorumDF = vecModel.transform(yorumDF)

yorumDF = yorumDF.select('features','label')
trainDF,testDF = yorumDF.randomSplit([0.75,0.25],seed=1234)

classifer = MultilayerPerceptronClassifier(layers=[kelimeSayisi,200,2])

model = classifer.fit(trainDF)
sonucDF = model.transform(testDF)

eva = BinaryClassificationEvaluator()
basari = eva.evaluate(sonucDF)
print(basari)

0.9026461538461539


In [11]:
yeniDF = spark.read.option('inferSchema','true')\
                        .option('header','false')\
                        .option('delimiter','\t').csv('datasets/deneme-yorum.txt')
yeniDF = yeniDF.withColumnRenamed('_c0','cumle')
yeniDF = tknzr.transform(yeniDF)
yeniDF = vecModel.transform(yeniDF)
yeniDF.show()
yeniYorumSonucDF = model.transform(yeniDF)
yeniYorumSonucDF.select('cumle','prediction').show()

+--------------------+--------------------+--------------------+
|               cumle|           tokenized|            features|
+--------------------+--------------------+--------------------+
| bu film çok kötüydü|[bu, film, çok, k...|(7563,[1,2,3,350]...|
|     berbat bir film| [berbat, bir, film]|(7563,[0,1,68],[1...|
|bundan daha güzel...|[bundan, daha, gü...|(7563,[13,376,161...|
| fena bir film değil|[fena, bir, film,...|(7563,[0,1,400],[...|
|çok kötü bir film...|[çok, kötü, bir, ...|(7563,[0,1,2,19,4...|
+--------------------+--------------------+--------------------+

+--------------------+----------+
|               cumle|prediction|
+--------------------+----------+
| bu film çok kötüydü|       0.0|
|     berbat bir film|       0.0|
|bundan daha güzel...|       1.0|
| fena bir film değil|       0.0|
|çok kötü bir film...|       0.0|
+--------------------+----------+

